# Reusing Pretrained Layers
*transfer learning*- using an already trained DNN layers except for the top ones. involves finding an existing neural network that accomplishes a similar task to the one you are trying to tackle

let'a look at an example:

In [6]:
# create a sample model
import tensorflow as tf
import numpy as np

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A.keras")

Epoch 1/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 540us/step - accuracy: 0.4978 - loss: 1.5797 - val_accuracy: 0.7909 - val_loss: 0.6877
Epoch 2/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 476us/step - accuracy: 0.8030 - loss: 0.6370 - val_accuracy: 0.8391 - val_loss: 0.5112
Epoch 3/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 478us/step - accuracy: 0.8425 - loss: 0.4984 - val_accuracy: 0.8546 - val_loss: 0.4393
Epoch 4/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 449us/step - accuracy: 0.8603 - loss: 0.4332 - val_accuracy: 0.8646 - val_loss: 0.3978
Epoch 5/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 461us/step - accuracy: 0.8700 - loss: 0.3934 - val_accuracy: 0.8714 - val_loss: 0.3707
Epoch 6/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 462us/step - accuracy: 0.8772 - loss: 0.3664 - val_accuracy: 0.8769 - val_loss: 0.3514
Epoch 7/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 426us/step - accuracy: 0.8831 - loss: 0.3465 - val_accuracy: 0.8802 - val_loss: 0.3369
Epoch 8/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 469us/step - accuracy: 0.8874 -